<a href="https://colab.research.google.com/github/hanialsab3/HM_Copy_of_287/blob/main/HM_Copy_of_287_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import os, cv2, dlib, glob
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.utils import plot_model
from keras.models import load_model
from keras.layers import Convolution2D, MaxPooling2D , ZeroPadding2D
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading Data**

In [ ]:
!unzip "drive/MyDrive/fer2013.zip" -d "drive/MyDrive"

Archive:  drive/MyDrive/fer2013.zip
replace drive/MyDrive/data/test/0/32298.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Reading the Data
# !unzip "drive/MyDrive/fer2013.zip" -d "drive/MyDrive"


trainpath = 'drive/MyDrive/data/train/'
testpath = 'drive/MyDrive/data/test/'
valpath = 'drive/MyDrive/data/val/'
# !pwd

row, col = 48, 48
classes = 7

def count_exp(path, set_):
    dict_ = {}
    for expression in os.listdir(path):
        dir_ = path + expression
        dict_[expression] = len(os.listdir(dir_))
    df = pd.DataFrame(dict_, index=[set_])
    return df
train_count = count_exp(trainpath, 'train')
test_count = count_exp(testpath, 'test')
val_count = count_exp(valpath, 'validation')
print(train_count)
print(test_count)
print(val_count)

          0    1     2     3     4     5     6
train  3995  436  4097  7215  4830  3171  4965
        0   1    2    3    4    5    6
test  491  55  528  879  594  416  626
              0   1    2    3    4    5    6
validation  467  56  496  895  653  415  607


**Loading Detector**

In [ ]:
# detector = dlib.get_frontal_face_detector()

In [ ]:
num_classes=7
img_rows,img_cols=48,48
batch_size=32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
 trainpath,
 color_mode='grayscale',
 target_size=(img_rows,img_cols),
 batch_size=batch_size,
 class_mode='categorical',
 shuffle=True)
validation_generator = validation_datagen.flow_from_directory(
 valpath,
 color_mode='grayscale',
 target_size=(img_rows,img_cols),
 batch_size=batch_size,
 class_mode='categorical',
 shuffle=True)
test_generator = test_datagen.flow_from_directory(
 testpath,
 color_mode='grayscale',
 target_size=(img_rows,img_cols),
 batch_size=batch_size,
 class_mode='categorical',
 shuffle=True)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [ ]:
img_rows,img_cols=48,48
num_classes=7
batch_size=32
# def cnn(weights_path=None, shape=(48, 48)):
#   model = Sequential()
#   model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
#                  input_shape=(1,img_rows,img_cols)))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(Conv2D(32,3,3,padding='same',kernel_initializer='he_normal',
#                  input_shape=(1,img_rows,img_cols)))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(MaxPooling2D(pool_size=(2,2)))
#   model.add(Dropout(0.2))
#   #Block-2
#   model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(MaxPooling2D(pool_size=(2,2)))
#   model.add(Dropout(0.2))
#   #Block-3
#   model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(MaxPooling2D(pool_size=(2,2)))
#   model.add(Dropout(0.2))
#   #Block-4
#   model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(MaxPooling2D(pool_size=(2,2)))
#   model.add(Dropout(0.2))
#   #Block-5
#   model.add(Flatten())
#   model.add(Dense(64,kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.5))
#   #Block-6
#   model.add(Dense(64,kernel_initializer='he_normal'))
#   model.add(Activation('relu'))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.5))
#   #Block-7
#   model.add(Dense(num_classes,kernel_initializer='he_normal'))
#   model.add(Activation('softmax'))
#   if weights_path:
#     model.load_weights(weights_path)
   
#   model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
#   model_json = model.to_json()
#   with open("model.json", "w") as json_file: json_file.write(model_json)
#   return model

In [ ]:
# model = cnn()
# model.summary()

In [ ]:
def cnn(weights_path=None, shape=(48, 48)):
  model = Sequential()
  model.add(ZeroPadding2D((1,1), input_shape=(48,48,1))) 
  model.add(Convolution2D(32, 3, 3, activation='relu')) 
  model.add(BatchNormalization())
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(32, 3, 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))
  model.add(Dropout(0.2))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(64, 3, 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(64, 3, 3, activation='relu')) 
  model.add(BatchNormalization())  
  model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))
  model.add(Dropout(0.2))
  model.add(ZeroPadding2D((1,1))) 
  model.add(Convolution2D(128, 3, 3, activation='relu')) 
  model.add(BatchNormalization())
  model.add(ZeroPadding2D((1,1))) 
  model.add(Convolution2D(128, 3, 3, activation='relu')) 
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))
  model.add(Dropout(0.2))
  model.add(ZeroPadding2D((1,1))) 
  model.add(Convolution2D(256, 3, 3, activation='relu')) 
  model.add(BatchNormalization())
  model.add(ZeroPadding2D((1,1))) 
  model.add(Convolution2D(256, 3, 3, activation='relu'))
  model.add(BatchNormalization()) 
  model.add(ZeroPadding2D((1,1))) 
  model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))
  model.add(Dropout(0.2)) 
  model.add(ZeroPadding2D((1,1))) 
  model.add(Convolution2D(512, 3, 3, activation='relu')) 
  model.add(BatchNormalization())
  model.add(ZeroPadding2D((1,1))) 
  model.add(Convolution2D(512, 3, 3, activation='relu')) 
  model.add(BatchNormalization())
  model.add(ZeroPadding2D((1,1))) 
  model.add(Convolution2D(512, 3, 3, activation='relu')) 
  model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))
  # model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(64, activation='relu')) 
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu')) 
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  print ("Create model successfully")
  if weights_path:
    model.load_weights(weights_path)
   
  # model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  # model_json = model.to_json()
  # with open("model.json", "w") as json_file: json_file.write(model_json)
  return model

In [ ]:
model = cnn()
model.summary()

Create model successfully
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 32)        128       
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 32)          128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (

In [ ]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
callbacks = [earlystop,checkpoint,reduce_lr]

In [ ]:
model.compile(loss='categorical_crossentropy',
 optimizer = Adam(lr=0.001),
 metrics=['accuracy'])

nb_train_samples = 28709
nb_validation_samples = 3589

In [ ]:
nb_validation_samples//batch_size


112

In [ ]:
nb_train_samples//batch_size

897

In [ ]:
history = model.fit_generator(
 train_generator,
 verbose=2,
 epochs = 3,
 callbacks = callbacks,
 validation_data = validation_generator,
 validation_steps = round(nb_validation_samples)//batch_size,
 steps_per_epoch = round(nb_train_samples)//batch_size,)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
897/897 - 578s - loss: 1.8141 - accuracy: 0.2437 - val_loss: 1.7967 - val_accuracy: 0.2489

Epoch 00001: val_loss did not improve from 1.79220
Epoch 2/3
897/897 - 580s - loss: 1.8107 - accuracy: 0.2445 - val_loss: 1.7897 - val_accuracy: 0.2497

Epoch 00002: val_loss improved from 1.79220 to 1.78970, saving model to EmotionDetectionModel.h5
Epoch 3/3
897/897 - 564s - loss: 1.8112 - accuracy: 0.2439 - val_loss: 1.7961 - val_accuracy: 0.2480

Epoch 00003: val_loss did not improve from 1.78970


In [ ]:
gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
faces=face_classifier.detectMultiScale(gray,1.3,5)

NameError: ignored

In [ ]:
for (x,y,w,h) in faces:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray=gray[y:y+h,x:x+w]
    roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

In [ ]:
if np.sum([roi_gray])!=0:
   roi=roi_gray.astype('float')/255.0
   roi=img_to_array(roi)
   roi=np.expand_dims(roi,axis=0)
   preds=classifier.predict(roi)[0]
   label=class_labels[preds.argmax()]
   label_position=(x,y)
   cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
else:
   cv2.putText(frame,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
cv2.imshow('Emotion Detector',frame)
if cv2.waitKey(1) & 0xFF == ord('q'): 
   break

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
while True:
    ret,frame=cap.read()
    labels=[]
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray=gray[y:y+h,x:x+w]
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi=roi_gray.astype('float')/255.0
            roi=img_to_array(roi)
            roi=np.expand_dims(roi,axis=0)

            preds=classifier.predict(roi)[0]
            label=class_labels[preds.argmax()]
            label_position=(x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,'No Face Found',(20,20),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# from google.colab.patches import cv2_imshow
# from skimage import io 
# read the images in the training dataset

# for images in os.listdir(pathtrain):
#   image = cv2.imread(images)
#   # gray = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY)
#   cv2_imshow(image)

# def load_images(folder):
#   images = []
#   for image in os.listdir(folder):
#     img = cv2.imread(os.path.join(folder,image))
#     if img is not None:
#       images.append(img)
#   return images

# tr = load_images(pathtrain)

# for image in tr:
#   cv2_imshow(image)


# images = [cv2.imread(file) for file in glob.glob(pathtrain)]

In [ ]:
# train_count.transpose().plot(kind='bar')

In [ ]:
# test_count.transpose().plot(kind='bar')

In [ ]:
# plt.figure(figsize=(14,22))
# i = 1
# for expression in os.listdir(pathtrain):
#     img = load_img((pathtrain + expression +'/'+ os.listdir(pathtrain + expression)[1]))
#     plt.subplot(1,7,i)
#     plt.imshow(img)
#     plt.title(expression)
#     plt.axis('off')
#     i += 1
# plt.show()

In [ ]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    zoom_range=0.3,
#                                    horizontal_flip=True)

# training_set = train_datagen.flow_from_directory(pathtrain,
#                                                 batch_size=64,
#                                                 target_size=(48,48),
#                                                 shuffle=True,
#                                                 color_mode='grayscale',
#                                                 class_mode='categorical')

# test_datagen = ImageDataGenerator(rescale=1./255)
# test_set = test_datagen.flow_from_directory(pathtest,
#                                                 batch_size=64,
#                                                 target_size=(48,48),
#                                                 shuffle=True,
#                                                 color_mode='grayscale',
#                                                 class_mode='categorical')

In [ ]:
# training_set.class_indices

In [ ]:
# def get_model(input_size, classes=7):
#      #Initialising the CNN
#     model = tf.keras.models.Sequential()   

#     model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape =input_size))
#     model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(2, 2))
#     model.add(Dropout(0.25))

#     model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))
#     model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))

#     model.add(Flatten())
#     model.add(Dense(1024, activation='relu'))
#     model.add(Dropout(0.5))
    
#     model.add(Dense(classes, activation='softmax'))

#     #Compliling the model
#     model.compile(optimizer=Adam(lr=0.0001, decay=1e-6), 
#                   loss='categorical_crossentropy', 
#                   metrics=['accuracy'])
#     return model

In [ ]:
# fernet = get_model((row,col,1), classes)
# fernet.summary()

In [ ]:
# plot_model(fernet, to_file='fernet.png', show_shapes=True, show_layer_names=True)

In [ ]:
# chk_path = 'ferNet.h5'
# log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# checkpoint = ModelCheckpoint(filepath=chk_path,
#                              save_best_only=True,
#                              verbose=1,
#                              mode='min',
#                              moniter='val_loss')

# earlystop = EarlyStopping(monitor='val_loss', 
#                           min_delta=0, 
#                           patience=3, 
#                           verbose=1, 
#                           restore_best_weights=True)
                        
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
#                               factor=0.2, 
#                               patience=6, 
#                               verbose=1, 
#                               min_delta=0.0001)


# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# csv_logger = CSVLogger('training.log')

# callbacks = [checkpoint, reduce_lr, csv_logger]

In [ ]:
# steps_per_epoch = training_set.n // training_set.batch_size
# validation_steps = test_set.n // test_set.batch_size

# hist = fernet.fit(x=training_set,
#                  validation_data=test_set,
#                  epochs=60,
#                  callbacks=callbacks,
#                  steps_per_epoch=steps_per_epoch,
#                  validation_steps=validation_steps)